In [237]:
import re
import jieba
import jieba.analyse
import json
import random
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

In [238]:
from elasticsearch import Elasticsearch
import elasticsearch.helpers
from datetime import datetime

In [239]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [240]:
# Find top10
res = es.search(index='facebook', doc_type='user', body= {'query': {'match': {'gender': 'female'}}})
print("%d documents found" % res['hits']['total'])

58 documents found


In [241]:
# Take all result
users = list(elasticsearch.helpers.scan(es, index='facebook', doc_type='user'))
total_user = [(user['_id'], user['_source'].get('gender')) for user in users]
print(len(total_user))

143


In [242]:
no_message = ['1618047998214350', '1624029027647185', '1454298984658260']
foreigners = ['1695358370488750', '10213401737335928', '1532260126834371', '10154597524691479', '10155404471078548', '10158882655745234']

In [243]:
user_dict = {'male': [], 'female': []}
for user in total_user:
    if user[0] not in (no_message + foreigners):
        if user[1] == 'male':
            user_dict['male'].append(user[0])
        else:
            user_dict['female'].append(user[0])

In [244]:
print('male: ' + str(len(user_dict['male'])) + '\nfemale: ' + str(len(user_dict['female'])))

male: 80
female: 54


In [278]:
# Filtering urls
def filtering_url(messages):
    filtered_messages = [re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', message) for message in messages]
    return filtered_messages

In [279]:
# Mapping user posts
def get_posts(users):
    posts_dic = {}
    for user in users:
        # print(user)
        posts = list(elasticsearch.helpers.scan(es, query={'query': {'match': {'user_id': user}}}, index='facebook', doc_type='post'))
        messages = [post['_source'].get('message') for post in posts if post['_source'].get('message') != '']
        #print(message)
        filtered_messages = filtering_url(messages)
        posts_dic[user] = filtered_messages
    return posts_dic

In [280]:
male_posts_dict = get_posts(user_dict['male'])
female_posts_dict = get_posts(user_dict['female'])

In [257]:
# Check if content is Chinese or not
def is_Chinese(uchar):
    if uchar >= u'\u4e00' and uchar<=u'\u9fa5':  
        return True  
    else:  
        return False

In [282]:
# Using jieba to tokenize user posts
def tokenize_posts(user_posts):
    seg_posts = {}
    for key, messages in user_posts.items():
        seg_posts[key] = [" ".join(jieba.cut(message, cut_all = False)) for message in messages]
    return seg_posts

In [283]:
male_seg_posts = tokenize_posts(male_posts_dict)
female_seg_posts = tokenize_posts(female_posts_dict)

In [284]:
# Display tfidf score
def display_scores(vectorizer, tfidf_result):
    # http://stackoverflow.com/questions/16078015/
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    index_scores = []
    for index, item in enumerate(sorted_scores):
        index_scores.append((index, item[1]))
    return sorted_scores

In [285]:
def calculate_tfidf(seg_posts):
    corpus = [segment for key, segments in seg_posts.items() for segment in segments]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf = tfidf_vectorizer.fit_transform(corpus)
    tfidf_features = tfidf_vectorizer.get_feature_names()
    
    sorted_scores = display_scores(tfidf_vectorizer, tfidf)
    return sorted_scores

In [286]:
male_tfidf_scores = calculate_tfidf(male_seg_posts)
female_tfidf_scores = calculate_tfidf(female_seg_posts)

In [287]:
male_bag_of_words = [word[0] for word in male_tfidf_scores[:100]]
female_bag_of_words = [word[0] for word in female_tfidf_scores[:100]]

In [288]:
male_num = 0
for key, messages in male_posts_dict.items():
    male_num += len(messages)

In [221]:
male_num

62217

In [222]:
female_num = 0
for key, messages in female_posts_dict.items():
    female_num += len(messages)

In [223]:
female_num

23535